In [54]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql import functions as f

StatementMeta(tcccibele, 11, 54, Finished, Available)

In [55]:
# Definir schema do arquivo
schema = StructType([ 
    StructField("Ano", StringType(), True), 
    StructField("Trimestre", StringType(), True), 
    StructField("Mes", StringType(), True), 
    StructField("Dia", StringType(), True), 
    StructField("TipoViolacao", StringType(), True), 
    StructField("GrupoVulneravel", StringType(), True), 
    StructField("EspecieViolacao", StringType(), True), 
    StructField("PaisOrigemVitima", StringType(), True), 
    StructField("UF", StringType(), True), 
    StructField("CanalAtendimento", StringType(), True), 
    StructField("DenunciaEmergencial", StringType(), True), 
    StructField("Denunciante", StringType(), True), 
    StructField("Motivacao", StringType(), True), 
    StructField("CenarioViolacao", StringType(), True), 
    StructField("Agravantes", StringType(), True), 
    StructField("AgravantesPoliciais", StringType(), True),
    StructField("RelacaoVitimaSuspeito", StringType(), True),
    StructField("SexoVitima", StringType(), True),
    StructField("IdadeVitima", StringType(), True),
    StructField("EscolaridadeVitima", StringType(), True),
    StructField("RacaVitima", StringType(), True),
    StructField("FaixaRendaVitima", StringType(), True),
    StructField("NacionalidadeVitima", StringType(), True),
    StructField("DeficienciaVitima", StringType(), True),
    StructField("RelacaoDemandanteVitima", StringType(), True),
    StructField("SexoSuspeito", StringType(), True),
    StructField("IdadeSuspeito", StringType(), True),
    StructField("EscolaridadeSuspeito", StringType(), True),
    StructField("NacionalidadeSuspeito", StringType(), True),
    StructField("RacaSuspeito", StringType(), True),
    StructField("FaixaRendaSuspeito", StringType(), True),
    StructField("PaisOrigemSuspeito", StringType(), True),
    StructField("DeficienciaSuspeito", StringType(), True)
])

StatementMeta(tcccibele, 11, 55, Finished, Available)

In [56]:
# Ler arquivo de origem da landing zone
df = spark.read.format("csv") \
            .schema(schema) \
            .options(quote='"', sep=';', encoding='ISO-8859-1') \
            .load('/landing_zone/2020/primeiro_semestre_2020.csv')

StatementMeta(tcccibele, 11, 56, Finished, Available)

In [57]:
# Remover colunas e linhas sem valor para o data lake
df = df.where((df[0].isNotNull()) \
                & (df[0] != 'Ano'))

StatementMeta(tcccibele, 11, 57, Finished, Available)

In [58]:
# Capitalizar todos os textos e incluir 0 onde há nulos
names = df.schema.names
for name in names:
    df = df.withColumn(name, f.initcap(f.col(name)))
    #df = df.withColumn(name,f.regexp_replace(name,',',''))
    #df = df.withColumn(name, df[name].cast(IntegerType()))

df = df.fillna(0)

StatementMeta(tcccibele, 11, 58, Finished, Available)

In [59]:
df.limit(3).toPandas()

StatementMeta(tcccibele, 11, 59, Finished, Available)

,Ano,Trimestre,Mes,Dia,TipoViolacao,GrupoVulneravel,EspecieViolacao,PaisOrigemVitima,UF,CanalAtendimento,DenunciaEmergencial,Denunciante,Motivacao,CenarioViolacao,Agravantes,AgravantesPoliciais,RelacaoVitimaSuspeito,SexoVitima,IdadeVitima,EscolaridadeVitima,RacaVitima,FaixaRendaVitima,NacionalidadeVitima,DeficienciaVitima,RelacaoDemandanteVitima,SexoSuspeito,IdadeSuspeito,EscolaridadeSuspeito,NacionalidadeSuspeito,RacaSuspeito,FaixaRendaSuspeito,PaisOrigemSuspeito,DeficienciaSuspeito
0,2020,Trim 1,Janeiro,1,2. Violência Psicológica,1-violência Contra A Mulher,Tortura Psíquica,Brasil,Al,Denúncia - Atendimento Telefônico,Não,O Próprio Agressor,N/d,Via Pública,Em Público - Outras Pessoas Presenciam As Agre...,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,Masculino,37 Anos,Ensino Fundamental Incompleto,Brasileiro(a),Parda,Até 1 Salário Mínimo,Brasil,Não
1,2020,Trim 1,Janeiro,1,2. Violência Psicológica,1-violência Contra A Mulher,Ameaça/coação,Brasil,Al,Denúncia - Atendimento Telefônico,Não,O Próprio Agressor,Em Razão Do Sexo Biológico,Via Pública,Resultado Das Agressões Se Prolongam Ao Longo ...,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,Masculino,37 Anos,Ensino Fundamental Incompleto,Brasileiro(a),Parda,Até 1 Salário Mínimo,Brasil,Não
2,2020,Trim 1,Janeiro,1,7. Violações Gerais,1-violência Contra A Mulher,Liberdade Sexual Psíquica - Abuso Sexual Psíquico,Brasil,Al,Denúncia - Atendimento Telefônico,Não,O Próprio Agressor,Em Razão Do Sexo Biológico,Via Pública,Resultado Das Agressões Se Prolongam Ao Longo ...,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,N/d,Masculino,37 Anos,Ensino Fundamental Incompleto,Brasileiro(a),Parda,Até 1 Salário Mínimo,Brasil,Não


In [60]:
# Escrever dados limpos em formato parque no azure storage
#df.write.format('parquet').mode('overwrite').save('/history_zone/2020/PrimeiroSemestre/')

df.write.partitionBy("Ano","Mes").mode("overwrite").parquet("/history_zone/2020/PrimeiroSemestre/")


StatementMeta(tcccibele, 11, 60, Finished, Available)

In [61]:
df = spark.read.parquet('/history_zone/2020/PrimeiroSemestre/')
df.limit(3).toPandas()

StatementMeta(tcccibele, 11, 61, Finished, Available)

,Trimestre,Dia,TipoViolacao,GrupoVulneravel,EspecieViolacao,PaisOrigemVitima,UF,CanalAtendimento,DenunciaEmergencial,Denunciante,Motivacao,CenarioViolacao,Agravantes,AgravantesPoliciais,RelacaoVitimaSuspeito,SexoVitima,IdadeVitima,EscolaridadeVitima,RacaVitima,FaixaRendaVitima,NacionalidadeVitima,DeficienciaVitima,RelacaoDemandanteVitima,SexoSuspeito,IdadeSuspeito,EscolaridadeSuspeito,NacionalidadeSuspeito,RacaSuspeito,FaixaRendaSuspeito,PaisOrigemSuspeito,DeficienciaSuspeito,Ano,Mes
0,Trim 1,9,1. Violência Física,2-violência Doméstica E Familiar Contra A Mulher,Maus Tratos,Brasil,Df,Denúncia - Atendimento Telefônico,Não,Anônimo,N/d,Casa Onde Reside A Vítima E O Suspeito,N/d,N/d,Companheiro(a),Feminino,21,Ensino Médio Completo,Preta,N/d,Brasileiro(a),Não,N/d,Masculino,24,Ensino Fundamental Incompleto,Brasileiro(a),Parda,N/d,N/d,Não,2020,Janeiro
1,Trim 1,9,1. Violência Física,2-violência Doméstica E Familiar Contra A Mulher,Tortura Física,Brasil,Df,Denúncia - Atendimento Telefônico,Não,Anônimo,N/d,Casa Onde Reside A Vítima E O Suspeito,N/d,N/d,Companheiro(a),Feminino,21,Ensino Médio Completo,Preta,N/d,Brasileiro(a),Não,N/d,Masculino,24,Ensino Fundamental Incompleto,Brasileiro(a),Parda,N/d,N/d,Não,2020,Janeiro
2,Trim 1,9,2. Violência Psicológica,2-violência Doméstica E Familiar Contra A Mulher,Tortura Psíquica,Brasil,Df,Denúncia - Atendimento Telefônico,Não,Anônimo,N/d,Casa Onde Reside A Vítima E O Suspeito,N/d,N/d,Companheiro(a),Feminino,21,Ensino Médio Completo,Preta,N/d,Brasileiro(a),Não,N/d,Masculino,24,Ensino Fundamental Incompleto,Brasileiro(a),Parda,N/d,N/d,Não,2020,Janeiro
